In [ ]:
    !pip install -q datasets 
!pip install -q transformers
!pip install torch

     |████████████████████████████████| 312 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 35.0 MB/s 
     |████████████████████████████████| 134 kB 38.4 MB/s 
     |████████████████████████████████| 67 kB 2.2 MB/s 
     |████████████████████████████████| 212 kB 37.2 MB/s 
     |████████████████████████████████| 127 kB 37.5 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 38.5 MB/s 
     |████████████████████████████████| 271 kB 38.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 39.4 MB/s 
     |████████████████████████████████| 6.5 MB 34.6 MB/s 
     |████████████████████████████████| 596

In [ ]:
import numpy as np
import random
import pandas as pd
import torch

from datasets import load_dataset
from datasets import load_metric
from datasets import Dataset
from transformers import pipeline
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

from sklearn.calibration import calibration_curve
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt

In [ ]:
# Download Berkeley Dataset on hate speech
dataset = load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
ucb = dataset['train'].to_pandas()
#ethos = pd.read_csv('/content/Ethos_Dataset_Multi_Label.csv', on_bad_lines='skip', sep=";")
#ghc = pd.read_csv("/content/ghc_train.tsv", sep='\t', header=0)

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/14.1M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Selectivly sort through dataset for sentances related to violence
data = ucb[["text", "violence"]]
data.drop_duplicates(subset=['text'], inplace=True)
pd.set_option('display.max_colwidth',1000)
display(data[data['violence'] == 3])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,text,violence
52,get the fuck out of my country before i bash you beta cunt.,3.0
54,Fuck Yeah ! Filthy Fuck'n Dirty Cock Cleanin' Sweat Fuck'n Drippin' Total Fuck'n Bottom Slut Christian Gettin' Gang Fuck'n Bred Hard and Fuck'n Rough for Balls Deep Fuck'n Loads of Filthy Fuck'n Dirty Cum Floodin' His Greedy Slutted Out Fuck'n Cunt (T.I.M)(Final Part) 👍 URL,3.0
69,"Are you people actually believing this propaganda..... These ratchet women haven't done a thing , except spew hate and rhetoric....they are gonna get Saddamed...",3.0
91,Ainz Ooal Pwn 229 yup 150 years after slavery was outlawed and still blacks sold other blacks into slavery what a shame,3.0
95,Washington Post and there white genocide agenda.,3.0
...,...,...
91658,Fuck that nigga him and his brother click baiting ass nigga,3.0
92042,Smd bitch,3.0
92203,I just bodied your fat mother. Cum all over that bitches face.,3.0
99927,"At 7:19 is why I hate people, women especially look how mad she got just because the dog splashed her.. f*** you you stupid b**** either have fun or go kill yourself",3.0


In [ ]:
# Simplify classification of violent material
data['violence'] = data['violence'].apply(lambda x: 1 if x > 2 else 0)
data = data.rename(columns={"violence":"labels"})
data.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,labels
count,39565.000000
mean,0.113509
std,0.317218
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
#Save data as CSV to download
data.to_csv("cs152_data.csv")

In [ ]:
# Create testing and training dataset
v_msk = np.random.rand(len(data[data['labels'] == 1])) < 0.90
n_msk = np.random.rand(len(data[data['labels'] == 0])) < 0.90
#[0:sum(v_msk)]
#[0:sum(~v_msk)]
train = data[data['labels'] == 1][v_msk].append(data[data['labels'] == 0][n_msk], ignore_index=True)
test = data[data['labels'] == 1][~v_msk].append(data[data['labels'] == 0][~n_msk], ignore_index=True)
print(len(test), len(train))
print(len(test[test['labels'] == 1])/ len(test), len(train[train['labels'] == 1])/ len(train))

4029 35536
0.1166542566393646 0.11315285907248987


In [ ]:
print(train)

                                                                                                                                                                                                                                                                                                                                                                                         text  \
0                                                                                                                                                                                                                                                                For starters bend over the one in pink and kick that ass and pussy to get a taste until she's begging for a dick inside her.   
1                                                                                                                                                                                                                                     

In [ ]:
#Make datasets hugging face daataaset
hf_test = Dataset.from_pandas(test).shuffle()
hf_train = Dataset.from_pandas(train).shuffle()
print(hf_train[4000])

{'text': 'We all know who did this 8the white people aka white police smh tge government hate our people', 'labels': 0}


In [ ]:
# Initialize and setup model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_test = hf_test.map(preprocess_function, batched=True)
tokenized_train = hf_train.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# Train Model
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", num_train_epochs=5)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_train, eval_dataset=tokenized_test, tokenizer=tokenizer, data_collator=data_collator)

trainer.train()
trainer.evaluate()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 35536
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 22210


Epoch,Training Loss,Validation Loss


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json


KeyboardInterrupt: ignored

In [ ]:
# Save Model Results
model.save_pretrained("Best_model")

In [ ]:
# Setup pipeline to run model
nlp = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer, device=0)
print(nlp)

In [ ]:
#Load our best model
first_model = AutoModelForSequenceClassification.from_pretrained('Best_model', num_labels=2)
nlp_2 = pipeline("sentiment-analysis",model=first_model, tokenizer=tokenizer)

In [ ]:
#Good Examples
print("Good Examples")
print(nlp_2("Please turn the lights on"))
print(nlp_2("I'm gonna eradicate them"))
print(nlp_2("they don't deserve to live"))
print(nlp_2("Gonna shoot a movie at school"))
print(nlp_2("My post is blowing up!"))
print(nlp_2("lets kill them all"))
#Bad Ecamples
print("Bad Examples")
print(nlp_2("Not even the police can stop me from killing now"))
print(nlp_2("I know where they live, let's make them regret it"))
print(nlp_2("Gonna light up the halls with my ak in an hour"))
print(nlp_2("bringing my gun to school tomorrow"))
print(nlp_2("he threw the first punch"))
print(nlp_2("Shoot towards the goal"))

In [ ]:
# Get Confusion Matrix
true_test_labels = tokenized_test['labels']        
def label_to_int(label):
  return 1 if label=="LABEL_1" else 0
predictions = []
for text in tokenized_test['text']:
  pred = nlp_2(text)[0]
  prediction = label_to_int(pred['label'])
  predictions.append(prediction)

from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(true_test_labels, predictions).ravel()
total = sum((tn, fp, fn, tp))
print((tn+tp)/total)
print(tn, fp, fn, tp)
print(tn/total, fp/total, fn/total, tp/total)

In [ ]:
true_test_labels = tokenized_test['labels']        
def pred_to_int(pred):
  return 1 if pred['label']=="LABEL_1" and pred['score'] > .98 else 0
predictions = []
for text in tokenized_test['text']:
  pred = nlp(text)[0]
  prediction = pred_to_int(pred)
  predictions.append(prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(true_test_labels, predictions).ravel()
total = sum((tn, fp, fn, tp))
print((tn+tp)/total)
print(tn/total, fp/total, fn/total, tp/total)

In [ ]:
true_test_labels = tokenized_test['labels']        
def label_to_int(label):
  return 1 if label=="LABEL_1" else 0
predictions = []
for text in tokenized_test['text']:
  pred = nlp(text)[0]
  prediction = label_to_int(pred['label'])
  predictions.append(prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(true_test_labels, predictions).ravel()
total = sum((tn, fp, fn, tp))
print((tn+tp)/total)
print(tn/total, fp/total, fn/total, tp/total)

In [ ]:
import matplotlib.pyplot as plt
predictions = []
for text in tokenized_test['text']:
  predictions.append(nlp(text)[0]['score'])

plt.figure()
plt.title('Scores: Total')
n, bins, patches = plt.hist(predictions, facecolor='blue', alpha = .5, bins = 50)
plt.show()  

In [ ]:
predictions.sort()
print(predictions)

In [ ]:
pr = np.array(predictions)
numNotSure = np.count_nonzero(pr < .98)

In [ ]:
print(numNotSure)
print(len(predictions))